<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Implementación de Word2vec con skip-grams<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2025-03-12</small></i></div>

***

## Introducción

Empezamos con lo más importante: las técnicas de _word embedding_ son una forma de representar numéricamente las palabras, pero con matices adicionales. Dicho esto, vamos a programar un proceso de aprendizaje de _embeddings_ a partir de texto. Nos centraremos en una técnica llamada **Word2Vec**, que ya tiene bastantes años pero que para pequeños embeddings sigue teniendo uso.

Word2Vec se basa en una red neuronal que genera la matriz mediante entrenamiento supervisado en un problema de clasificación. El artículo en el que se presenta el método es [_Efficient Estimation of Word Representations in Vector Space_ (Mikolov et al., 2013)](https://arxiv.org/pdf/1301.3781.pdf) y se utiliza con para medir la **similitud sintáctica y semántica de las palabras**.

El artículo explora dos modelos: _Continuous Bag-of-Words_ y _Skip-gram_. Este último es el más utilizado y será el que abordemos aquí.

La idea del _Skip-gram_ es la siguiente: dada una palabra (a la que llamaremos _palabra de contexto_), queremos entrenar un modelo que sea capaz de predecir una palabra que pertenezca a una ventana de tamaño $N$. Por ejemplo, asumiendo una ventana de tamaño $N = 3$ y dada la siguiente frase:

> All those <span style="color:red">moments will be</span> **lost** <span style="color:red">in time like</span> tears in rain

La _palabra de contexto_ sería **lost**, y entrenaríamos el modelo para que predijera una de las palabras existentes dentro de la ventana especificada, es decir, una de `['moments', 'will', 'be', 'in', 'time', 'like']`.

## Objetivos

En este notebook crearemos un _embedding_ utilizando la técnica _skip-gram_ de **Word2Vec**.

## Librerías y configuración

In [ ]:
import collections
import gzip
import os
import pathlib
import random
import re
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import torch

import utils

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (16, 9), 'figure.dpi': 100})

In [ ]:
EMBEDDING_DIM = 5
TRAIN_BATCH = 256
TRAIN_EPOCH = 50
WINDOW_SIZE = 5

MODELS_DIR = pathlib.Path('../Models')
MODELS_DIR.mkdir(exist_ok=True)
TEMP_PATH = pathlib.Path('tmp')
TEMP_PATH.mkdir(exist_ok=True)

SAVE_MODEL = True
LOAD_MODEL = True
MODEL_PATH = MODELS_DIR / "skipgrams.pt"

## Construcción del corpus

Utilizaremos un _dataset_ de reseñas de Amazon no muy actual, pero interesante para entrenar el modelo.

In [ ]:
DATASET_URL = 'https://github.com/blazaid/aprendizaje-profundo/raw/refs/heads/gh-pages/Datasets/Video_Games_5.json.gz'
DATASET = pathlib.Path('tmp/Video_Games_5.json')

print(f"Downloading dataset to {DATASET.resolve()}")
if not DATASET.exists():
    with requests.get(DATASET_URL, stream=True) as response:
        response.raise_for_status()
        with gzip.GzipFile(fileobj=response.raw) as f_gz:
            with DATASET.open("wb") as f:
                shutil.copyfileobj(f_gz, f)
else:
    print("File already exists! Nice")

print("Loading text corpus")
corpus = pd.read_json(DATASET, lines=True)
corpus = corpus['reviewText'].astype(str).str.strip()
corpus.head()

print("Done")

A continuación crearemos una clase que se encargará de realizar la tokenización de nuestros textos. La idea es que sea una clase que ajusta los textos pasados y convierta dichos textos a secuencias de indices a texto.

In [ ]:
class SimpleTokenizer:
    def __init__(self, unk_token="<UNK>"):
        self.word_index = {}
        self.index_word = {}
        self.unk_token = unk_token

    def fit_on_texts(self, texts, reset=True):
        counter = collections.Counter()
        for text in texts:
            words = re.findall(r'\b\w+\b', text.lower())
            counter.update(words)

        if reset:
            # Ajustamos exclusivamente a estos textos
            self.word_index = {word: i for i, (word, _) in enumerate(counter.most_common(), 1)}
            if self.unk_token not in self.word_index:
                self.word_index[self.unk_token] = len(self.word_index) + 1
        else:
            # Actualizamos los textos existentes
            current_max_index = max(self.word_index.values(), default=0)
            for word, _ in counter.most_common():
                if word not in self.word_index:
                    current_max_index += 1
                    self.word_index[word] = current_max_index
            if self.unk_token not in self.word_index:
                self.word_index[self.unk_token] = current_max_index + 1

        # En cualquier caso, actualizamos los índices
        self.index_word = {i: word for word, i in self.word_index.items()}

    def texts_to_sequences(self, texts):
        unk_index = self.word_index.get(self.unk_token)
        sequences = []
        for text in texts:
            words = re.findall(r'\b\w+\b', text.lower())
            seq = [self.word_index.get(word, unk_index) for word in words]
            sequences.append(seq)
        return sequences

tokenizer = SimpleTokenizer()
tokenizer.fit_on_texts([
    "I've seen things you people wouldn't believe",
    "Attack ships on fire off the shoulder of Orion",
    "I watched C-beams glitter in the dark near the Tannhäuser Gate",
])
tokenizer.texts_to_sequences([
    "All those moments will be lost in time, like tears in rain",
    "Time to die",
])

Bueno, parece que el índice asignado al «token desconocido» es 28. Esperemos que haya más variedad de palabras analizando el contenido de nuestro corpus.

La variable `corpus` contiene todas las reseñas. _Tokenizaremos_ cada comentario, convirtiéndolo en una lista de palabras, utilizando nuestro tokenizador.

In [ ]:
tokenizer = SimpleTokenizer()
tokenizer.fit_on_texts(corpus)

# Mostramos algunos de los primeros elementos de los diccionarios
print(f'word2id: {dict(list(tokenizer.word_index.items())[0:4])} ...')
print(f'id2word: {dict(list(tokenizer.index_word.items())[0:4])} ...')

Convertimos cada reseña en una secuencia de enteros utilizando el tokenizador.

In [ ]:
sentences = tokenizer.texts_to_sequences(corpus)
vocab_size = len(tokenizer.word_index) + 1

print(f'Corpus sentences: {len(sentences)} oraciones')
print(f'Vocabulary Size: {vocab_size} palabras')
print('Sentence example:')
print(f'- {corpus.iloc[5]}')
print(f'- {sentences[5]}')

## Generador de Skip-grams

Ahora, el siguiente paso es definir una función que nos genere los _skip-grams_. Esta función generará pares positivos (dentro de la ventana) y negativos (muestreo aleatorio).

In [ ]:
def skipgrams(sequence, vocabulary_size, window_size, negative_samples=1):
    pairs, labels = [], []
    for i, target in enumerate(sequence):
        window_start = max(0, i - window_size)
        window_end = min(len(sequence), i + window_size + 1)
        context_indices = [j for j in range(window_start, window_end) if j != i]
        for j in context_indices:
            context_word = sequence[j]
            pairs.append((target, context_word))
            labels.append(1)
            for _ in range(negative_samples):
                negative_word = random.randint(1, vocabulary_size - 1)
                if negative_word == context_word:
                    continue
                pairs.append((target, negative_word))
                labels.append(0)
    return pairs, labels

El proceso de cálculo de _skip-grams_ es **muy** pesado, tanto en tiempo como en espacio. Por lo tanto, crearemos un dataset que calculará el batch de skipgrams que toca en cada llamada de obtención de dicho batch.

In [ ]:
class SkipGramDataset(torch.utils.data.IterableDataset):
    def __init__(self, sentences, vocab_size, window_size):
        self.sentences = sentences
        self.vocab_size = vocab_size
        self.window_size = window_size
        self.total_samples = sum(
            len(skipgrams(
                sentence,
                vocabulary_size=vocab_size,
                window_size=window_size
            )[0])
            for sentence in sentences
        )

    def __iter__(self):
        for sentence in self.sentences:
            pairs, labels = skipgrams(sentence, vocabulary_size=self.vocab_size, window_size=self.window_size)
            for (target, context), label in zip(pairs, labels):
                target_tensor = torch.tensor(target, dtype=torch.long)
                context_tensor = torch.tensor(context, dtype=torch.long)
                label_tensor = torch.tensor(label, dtype=torch.float32)
                yield target_tensor, context_tensor, label_tensor

    def __len__(self):
        return self.total_samples


dataset = SkipGramDataset(sentences, vocab_size, WINDOW_SIZE)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=TRAIN_BATCH,
)  # Shuffle no funcina con un IterableDataset

## Creación y entrenamiento del modelo

Ya tenemos un dataset con las entradas y sus respectivas salidas. Ahora el objetivo es entrenar un modelo que sea capaz de determinar si dos palabras pertenecen al mismo contexto.

Para ello, crearemos una capa de _embedding_ que transforme las palabras en su vector de características. La similitud entre los embeddings se mide mediante la _cosine similarity_.

In [ ]:
class SkipGramModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate=0.25):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.dropout = torch.nn.Dropout(dropout_rate)
        self.linear = torch.nn.Linear(1, 1)
        
    def forward(self, target, context):
        # target y context tienen forma (batch,)
        target_emb = self.embedding(target)    # (batch, embedding_dim)
        target_emb = self.dropout(target_emb)
        context_emb = self.embedding(context)  # (batch, embedding_dim)
        context_emb = self.dropout(context_emb)
        
        similarity = nn.functional.cosine_similarity(target_emb, context_emb, dim=1).unsqueeze(1)
        
        out = self.linear(similarity)
        out = torch.sigmoid(output)
        return out

model = SkipGramModel(vocab_size, EMBEDDING_DIM)
if LOAD_MODEL and MODEL_PATH.exists():
    model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
print(model)

Ahora entrenamos el modelo con los _skip-grams_ generados. Este proceso puede tardar **mucho** dependiendo de la máquina.

In [ ]:
history = utils.train(
    model=model,
    train_loader=dataloader,
    n_epochs=2,
    criterion=torch.nn.BCELoss(),
    optimizer=torch.optim.Adam(model.parameters()),
    validation_split=0.1,
)
if SAVE_MODEL:
    torch.save(music_generator.state_dict(), MODEL_PATH)

Veamos el progreso del entrenamiento:

In [ ]:
pd.DataFrame(history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

## Embeddings

Una vez entrenado el modelo, disponemos de una matriz con los pesos de las características para cada palabra. Extraemos esta matriz y la mostramos en un dataframe.

In [ ]:
weights = model.embedding.weight.data.cpu().numpy()[1:]

df = pd.DataFrame(weights, index=list(tokenizer.index_word.values()))
df.head(10)

Realicemos una búsqueda de las palabras más similares a una dada utilizando, por ejemplo, la distancia Euclidiana de sus vectores.

In [ ]:
NUM_CLOSEST_WORDS = 10
WORD = 'man'

v1 = weights[tokenizer.word_index[WORD] - 1]
words = sorted(
    [word for word in tokenizer.word_index.keys()],
    key=lambda w: np.linalg.norm(v1 - weights[tokenizer.word_index[w]-1])
)
df.loc[words[:NUM_CLOSEST_WORDS + 1], :]

## Conclusiones

En resumen, hemos implementado un _embedding_ utilizando la técnica de _skip-grams_ de **Word2Vec** y hemos demostrado su efectividad para representar las palabras de forma más significativa en un espacio vectorial. Esta técnica es capaz de capturar la semántica de las palabras, representándolas en un espacio de dimensión inferior al que ocuparía una representación _one-hot_.

Cabe destacar que, aunque el proceso de preprocesamiento y entrenamiento se ha simplificado, existen muchas mejoras posibles (como preprocesamiento avanzado, mayor cantidad de negative sampling, etc.) y muchos _embeddings_ preentrenados disponibles para uso.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>